In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [102]:
periods = list(range(1, 6))
prev_period = dict(zip(periods, [5, 1, 2, 3, 4]))
time_h  = dict(zip(periods, ['00:00~06:00',
                             '06:00~09:00',
                             '09:00~15:00',
                             '15:00~18:00',
                             '18:00~00:00']))
hours   = dict(zip(periods, [6, 3, 6, 3, 6]))
demand  = dict(zip(periods, [15000, 30000, 25000, 40000, 27000])) 

gen_types = [1, 2, 3]
gen_qty = dict(zip(gen_types, [12, 10, 5]))
gen_min = dict(zip(gen_types, [850, 1250, 1500]))
gen_max = dict(zip(gen_types, [2000, 1750, 4000]))
gen_basecost  = dict(zip(gen_types, [1000, 2600, 3000]))
gen_hourcost  = dict(zip(gen_types, [2, 1.3, 3]))
gen_startcost = dict(zip(gen_types, [2000, 1000, 500]))

In [107]:
model = gp.Model('Tariff Rates')

# add vars
n = model.addVars(gen_types, periods,  # qty of generators of type i already on in period t
                  vtype=GRB.INTEGER,
                  name='n')
s = model.addVars(gen_types, periods,  # qty of generators of type i started up in period t 
                  vtype=GRB.INTEGER,
                  name='s')
x = model.addVars(gen_types, periods,  # total output of generators of type i
                  name='x')

for i in gen_types:
    for j in periods:
        n[i,j].setAttr('ub', gen_qty[i])        

# objective function
model.setObjective((gp.quicksum(gen_startcost[i]*s[i,j] 
                                for i in gen_types for j in periods)
                  + gp.quicksum(gen_basecost[i]*hours[j]*n[i,j]
                                for i in gen_types for j in periods)
                  + gp.quicksum(gen_hourcost[i]*hours[j]*(x[i,j] - gen_min[i]*n[i,j])
                                for i in gen_types for j in periods)))

# constraints
model.addConstrs((gp.quicksum(x[i,j] for i in gen_types) >= demand[j] 
                  for j in periods),
                 name='demand')

model.addConstrs((x[i,j] >= gen_min[i]*n[i,j] 
                 for i in gen_types for j in periods),
                name='min_output')

model.addConstrs((x[i,j] <= gen_max[i]*n[i,j] 
                 for i in gen_types for j in periods),
                name='max_output')

model.addConstrs((gp.quicksum(gen_max[i]*n[i,j] for i in gen_types) >= 1.15*demand[j] 
                 for j in periods),
                name='extra_demand')

model.addConstrs((s[i,j] >= n[i,j] - n[i,prev_period[j]]) 
                 for i in gen_types for j in periods)

model.update()
model.write('Tariff Rates.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 55 rows, 45 columns and 135 nonzeros
Model fingerprint: 0x08b3bbff
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 5e+04]
Found heuristic solution: objective 1279070.0000
Presolve time: 0.00s
Presolved: 55 rows, 45 columns, 135 nonzeros
Variable types: 0 continuous, 45 integer (0 binary)

Root relaxation: objective 9.855143e+05, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 985514.286    0    7 1279070.00 985514.286  23.0%     -    0s
H    0     0                    995430.80000 985514.286  1.00%     -    0s
H    0     0                

In [108]:
print('Total cost: ${:.2f}'.format(model.ObjVal))

for j in periods:
    print('\nPeriod {}'.format(j))
    for i in gen_types:
        if n[i,j].x > 0:
            print('{} generators type {} producing {:.0f} MW'.format(n[i,j].x, i, x[i,j].x))
        

Total cost: $988540.00

Period 1
12.0 generators type 1 producing 10200 MW
3.0 generators type 2 producing 4800 MW

Period 2
12.0 generators type 1 producing 16000 MW
8.0 generators type 2 producing 14000 MW

Period 3
12.0 generators type 1 producing 11000 MW
8.0 generators type 2 producing 14000 MW

Period 4
12.0 generators type 1 producing 21250 MW
9.0 generators type 2 producing 15750 MW
2.0 generators type 3 producing 3000 MW

Period 5
12.0 generators type 1 producing 11250 MW
9.0 generators type 2 producing 15750 MW


In [109]:
for j in periods:
    print('\nPeriod {}'.format(j))
    for i in gen_types:
        if s[i,j].x>0:
            print('Start {} generators of type {}'.format(s[i,j].x, i))


Period 1

Period 2
Start 5.0 generators of type 2

Period 3

Period 4
Start 1.0 generators of type 2
Start 2.0 generators of type 3

Period 5
